In [1]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [ ]:
dino_table = MyPyTable().load_from_file('input_data/dinosaurs.csv')
dino_table.remove_rows_with_missing_values()


y = dino_table.get_column(dino_table.column_names.index("name"))  # The 'survived' column is the target
X = [row[:dino_table.column_names.index("name")] + row[dino_table.column_names.index("name") + 1:] for row in dino_table.data]

tree_classifier = MyDecisionTreeClassifier()
tree_classifier.fit(X,y)
header = dino_table.column_names

tree_classifier.print_decision_rules(attribute_names=header)
tree_classifier.visualize_tree(
    "dino_tree.dot", "dino_tree", attribute_names=header
)

print(len(dino_table.data))
print(len(set(dino_table.get_column(dino_table.column_names.index("name")))))


IF occurrence_no == 130209.0 THEN class = Chaoyangsaurus.
IF occurrence_no == 130294.0 THEN class = Protarchaeopteryx.
IF occurrence_no == 130295.0 THEN class = Caudipteryx.
IF occurrence_no == 139242.0 THEN class = Gorgosaurus.
IF occurrence_no == 139250.0 THEN class = Gorgosaurus.
IF occurrence_no == 139251.0 THEN class = Centrosaurus.
IF occurrence_no == 139252.0 THEN class = Gorgosaurus.
IF occurrence_no == 139253.0 THEN class = Gorgosaurus.
IF occurrence_no == 139254.0 THEN class = Gorgosaurus.
IF occurrence_no == 139255.0 THEN class = Albertosaurus.
IF occurrence_no == 139256.0 THEN class = Albertosaurus.
IF occurrence_no == 139257.0 THEN class = Albertosaurus.
IF occurrence_no == 139258.0 THEN class = Albertosaurus.
IF occurrence_no == 139259.0 THEN class = Albertosaurus.
IF occurrence_no == 139260.0 THEN class = Isisaurus.
IF occurrence_no == 139261.0 THEN class = Daspletosaurus.
IF occurrence_no == 139262.0 THEN class = Daspletosaurus.
IF occurrence_no == 139290.0 THEN class =